In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
learning_rate = 0.001
training_epochs = 21
batch_size = 100

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'cnn_session_mode'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

## TensorBoard

In [4]:
mnist = keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [5]:
train_x = train_x.astype(np.float32) / 255
test_x = test_x.astype(np.float32) / 255

train_x = np.expand_dims(train_x,3)
test_X = np.expand_dims(test_x,3)

train_y = to_categorical(train_y, 10)
test_y = to_categorical(test_y, 10)


train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=70000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))

num_train_data = train_x.shape[0]
num_test_data = test_x.shape[0]

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28) (10000, 10)


In [6]:
class Model:
    def __init__(self):
        self.X = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.Y = tf.placeholder(tf.float32, [None,10])
        self.is_training = tf.placeholder(tf.bool, [])
    
    def create_model(self, reuse=False):
        self.model = self._model(self.X, self.is_training, reuse=reuse)
        
    def _model(self, x, is_training=False, reuse=False):
        with tf.variable_scope('model_layers', reuse=reuse):
            conv1 = tf.contrib.layers.conv2d(x, 32, 3, padding='SAME', scope="conv1", activation_fn=None)
            # (n, 28, 28, 1) => (n, 28, 28,32)
            batch1 = tf.contrib.layers.batch_norm(conv1, is_training=is_training, scope="batch1",activation_fn=tf.nn.relu)
            pool1 = tf.contrib.layers.max_pool2d(batch1, 2)
            # (n, 28, 28, 32) => (n, 14, 14,32)
            
            conv2 = tf.contrib.layers.conv2d(pool1, 64, 3, padding='SAME',scope='conv2', activation_fn=None)
            # (n, 14, 14, 32) => (n, 14, 14, 64)
            batch2 = tf.contrib.layers.batch_norm(conv2, is_training=is_training, scope="batch2", activation_fn=tf.nn.relu)
            pool2 = tf.contrib.layers.max_pool2d(batch2, 2)
            # (n, 14, 14, 64) => (n, 7, 7, 64)
            
            conv3 = tf.contrib.layers.conv2d(pool2, 128, 3, padding='SAME', scope='conv3', activation_fn=None)
            # (n, 7, 7, 64) => (n, 7, 7, 128)
            batch3 = tf.contrib.layers.batch_norm(conv3, is_training=is_training, scope='batch3', activation_fn=tf.nn.relu)
            
            flatten = tf.contrib.layers.flatten(batch3, scope='flatten')
            
            # fully-connected-layer
            fc_layer1 = tf.contrib.layers.fully_connected(flatten, 256, activation_fn=None, scope='fc_layer1')
            batch4 = tf.contrib.layers.batch_norm(fc_layer1, is_training=is_training, scope='batch4', activation_fn=tf.nn.relu)
            
            output = tf.contrib.layers.fully_connected(batch4, 10, activation_fn=None, scope='output_layer')
            
            return output

In [7]:
cnn = Model()
cnn.create_model()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
